# Brain Tumor Classification

This notebook implements a deep learning model for classifying brain tumors from MRI scans.

## Table of Contents
1. [Setup and Installation](#setup)
2. [Data Loading and Preprocessing](#data)
3. [Model Architecture](#model)
4. [Training](#training)
5. [Evaluation](#evaluation)
6. [Inference](#inference)

In [ ]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. Setup and Installation <a name='setup'/>

Make sure you have all the required packages installed. You can install them using:
```
pip install -r requirements.txt
```

## 2. Data Loading and Preprocessing <a name='data'/>

In [ ]:
# Constants
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 20

# Paths
TRAIN_DIR = '../data/raw/Training'
TEST_DIR = '../data/raw/Testing'

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

## 3. Model Architecture <a name='model'/>

In [ ]:
def create_model(input_shape=IMG_SIZE + (3,), num_classes=4):
    """Create a CNN model for brain tumor classification."""
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Dense Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## 4. Training <a name='training'/>

In [ ]:
# Initialize the model
model = create_model()
model.summary()

## 5. Evaluation <a name='evaluation'/>

In [ ]:
def plot_training_history(history):
    """Plot training and validation accuracy and loss."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(EPOCHS)
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.tight_layout()
    plt.show()

## 6. Inference <a name='inference'/>

In [ ]:
def predict_tumor(image_path, model):
    """Predict the class of a brain MRI image."""
    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=IMG_SIZE
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    class_names = ['Glioma', 'Meningioma', 'No tumor', 'Pituitary']
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"Predicted: {class_names[tf.argmax(score)]} with {100 * tf.reduce_max(score):.2f}% confidence")
    plt.axis('off')
    plt.show()